In [ ]:
import os
import datetime as dt  # Python standard library datetime  module
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import netCDF4 as nc
import xarray as xr
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import RFE
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import RandomizedSearchCV
from eofs.xarray import Eof

In [ ]:
#**parameters & hyperparameters

RSCV = True
path_output='output_path/output.nc'

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 300, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5,55, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [5, 10, 15, 25]
# Minimum number of samples required at each leaf node
min_samples_leaf = [4, 8, 12]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
#! wget https://zenodo.org/record/5465895/files/train_val.tar.gz

In [ ]:
## Extract data into your "local" drive on Google
#! tar -xvf train_val.tar.gz
## Check all files in your local environment
#print("============== Files =============")
#! ls

In [ ]:
# Utilities for normalizing the emissions data
min_co2 = 0.
max_co2 = 2400
def normalize_co2(data):
    return data / max_co2

def un_normalize_co2(data):
    return data * max_co2

min_ch4 = 0.
max_ch4 = 0.6
def normalize_ch4(data):
    return data / max_ch4

def un_normalize_ch4(data):
    return data * max_ch4

data_path = "./"

In [ ]:
from glob import glob

inputs = glob(data_path + "inputs_s*.nc")
SECONDS_IN_YEAR = 60*60*24*365 #s

fig, axes = plt.subplots(2, 2, figsize=(12,12))

for input in inputs:
    label=input.split('_')[1][:-3]
    X = xr.open_dataset(input)
    x = range(2015, 2101)

    weights = np.cos(np.deg2rad(X.latitude))
    
    axes[0, 0].plot(x, X['CO2'].data, label=label)
    axes[0, 0].set_ylabel("Cumulative anthropogenic CO2 \nemissions since 1850 (GtCO2)")
    axes[0, 1].plot(x, X['CH4'].data, label=label)
    axes[0, 1].set_ylabel("Anthropogenic CH4 \nemissions (GtCH4 / year)")
    # FIXME: Not sure where this factor of 1000 comes from...! Maybe the CEDS data is really g/m-2/s?
    axes[1, 0].plot(x, X['SO2'].weighted(weights).sum(['latitude', 'longitude']).data*SECONDS_IN_YEAR*1e-9, label=label)
    axes[1, 0].set_ylabel("Anthropogenic SO2 \nemissions (GtSO2 / year)")
    axes[1, 1].plot(x, X['BC'].weighted(weights).sum(['latitude', 'longitude']).data*SECONDS_IN_YEAR*1e-9, label=label)
    axes[1, 1].set_ylabel("Anthropogenic BC \nemissions (GtBC / year)")

axes[0, 0].set_title('CO2')
axes[0, 1].set_title('CH4')
axes[1, 0].set_title('SO2')
axes[1, 1].set_title('BC')
axes[0, 0].legend()
plt.tight_layout()

test_data_path='/test-data/inputs_ssp245-2.nc'

In [ ]:
# Get one combined historical + ssp585 + ssp126 + ssp370 timeseries for now
X = xr.concat([xr.open_dataset(data_path + 'inputs_historical.nc'), xr.open_dataset(data_path + 'inputs_ssp585.nc'),xr.open_dataset(data_path+ 'inputs_ssp126.nc'),xr.open_dataset(data_path+ 'inputs_ssp370.nc'),xr.open_dataset(data_path+ 'inputs_hist-aer.nc'),xr.open_dataset(data_path+ 'inputs_hist-GHG.nc')], dim='time').compute()

# Take the average member for the historical, ssp585, ssp126, ssp370, hist-aer, hist-ghg
Y = xr.concat([xr.open_dataset(data_path + 'outputs_historical.nc').mean(dim="member"), xr.open_dataset(data_path + 'outputs_ssp585.nc').mean(dim="member"),xr.open_dataset(data_path+ 'outputs_ssp126.nc').mean(dim="member"),xr.open_dataset(data_path+ 'outputs_ssp370.nc').mean(dim="member"),xr.open_dataset(data_path+ 'outputs_hist-aer.nc').mean(dim="member"),xr.open_dataset(data_path+ 'outputs_hist-GHG.nc').mean(dim="member")], dim='time').compute()


# Convert the precip values to mm/day
Y["pr"] *= 86400
Y["pr90"] *= 86400

X["time"]=np.arange(1, 424 + 165 + 165) ## 165+86+86+86+65+165+1
Y["time"]=np.arange(1, 424 + 165 + 165)
        

In [ ]:
## proj = ccrs.PlateCarree()
#fig = plt.figure(figsize=(18, 8))
#plt.subplot(221)
#np.log(X["SO2"].sel(time=1850)).plot(vmin=-27, vmax=-20)
## plt.gca().coastlines()
#plt.gca().set_title('1850')
#
#plt.subplot(222)
#np.log(X["SO2"].sel(time=1970)).plot(vmin=-27, vmax=-20)
## plt.gca().coastlines()
#plt.gca().set_title('1970')
#
#plt.subplot(223)
#np.log(X["SO2"].sel(time=2020)).plot(vmin=-27, vmax=-20)
## plt.gca().coastlines()
#plt.gca().set_title('2020')
#
#plt.subplot(224)
#m=np.log(X["SO2"].sel(time=2100)).plot(vmin=-27, vmax=-20)
## plt.gca().coastlines()
#plt.gca().set_title('2100')
#
#fig.subplots_adjust(right=0.8)
#cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
#cbar = fig.colorbar(m, cax=cbar_ax)
#cbar.set_label('Log anthropogenic SO2 emissions (log kg / m2 / s)')

In [ ]:
#fig = plt.figure(figsize=(12, 8))
#plt.subplot(221)
#Y["tas"].sel(time=2100).plot(vmin=0, cmap='Reds')
#plt.gca().set_title('Annual Mean')
#
#plt.subplot(222)
#Y["diurnal_temperature_range"].sel(time=2100).plot(vmin=-4, vmax=4, cmap='RdBu_r' )
#plt.gca().set_title('Mean diurnal range')
#
#plt.subplot(223)
##  Convert precip to mm/day
#Y["pr"].sel(time=2100).plot(vmin=-5, vmax=5, cmap='RdBu_r')
#plt.gca().set_title('Annual Mean')
#
#plt.subplot(224)
#Y["pr90"].sel(time=2100).plot(vmin=-5, vmax=5, cmap='RdBu_r')
#plt.gca().set_title('Annual 90th percentile')

In [ ]:
# Create an EOF solver to do the EOF analysis. Square-root of cosine of
# latitude weights are applied before the computation of EOFs.
bc_solver = Eof(X['BC'])

# Retrieve the leading EOF, expressed as the correlation between the leading
# PC time series and the input SST anomalies at each grid point, and the
# leading PC time series itself.
bc_eofs = bc_solver.eofsAsCorrelation(neofs=5)
bc_pcs = bc_solver.pcs(npcs=5, pcscaling=1)

In [ ]:
# Create an EOF solver to do the EOF analysis. Square-root of cosine of
# latitude weights are applied before the computation of EOFs.
so2_solver = Eof(X['SO2'])

# Retrieve the leading EOF, expressed as the correlation between the leading
# PC time series and the input SST anomalies at each grid point, and the
# leading PC time series itself.
so2_eofs = so2_solver.eofsAsCorrelation(neofs=5)
so2_pcs = so2_solver.pcs(npcs=5, pcscaling=1)


In [ ]:
# Convert the Principle Components of the aerosol emissions (calculated above) in to Pandas DataFrames
bc_df = bc_pcs.to_dataframe().unstack('mode')
bc_df.columns = [f"BC_{i}" for i in range(5)]

so2_df = so2_pcs.to_dataframe().unstack('mode')
so2_df.columns = [f"SO2_{i}" for i in range(5)]

In [ ]:
# Bring the emissions data back together again and normalise
leading_historical_inputs = pd.DataFrame({
    "CO2": normalize_co2(X["CO2"].data),
    "CH4": normalize_ch4(X["CH4"].data)
}, index=X["CO2"].coords['time'].data)

# Combine with aerosol EOFs
leading_historical_inputs=pd.concat([leading_historical_inputs, bc_df, so2_df], axis=1)

In [ ]:
def get_rmse(truth, pred):
    weights = np.cos(np.deg2rad(truth.lat))
    return np.sqrt(((truth-pred)**2).weighted(weights).mean(['lat', 'lon'])).data.mean()

In [ ]:
y_inp_tas=Y["tas"].stack(dim=["lat", "lon"])
y_inp_pr=Y["pr"].stack(dim=["lat", "lon"])
y_inp_pr90=Y["pr90"].stack(dim=["lat", "lon"])
y_inp_dtr=Y["diurnal_temperature_range"].stack(dim=["lat", "lon"])

In [ ]:
df_y_input_tas = pd.DataFrame(y_inp_tas.to_pandas())
df_y_input_pr = pd.DataFrame(y_inp_pr.to_pandas())
df_y_input_pr90 = pd.DataFrame(y_inp_pr90.to_pandas())
df_y_input_dtr = pd.DataFrame(y_inp_dtr.to_pandas())

Xy_train_tas_ = pd.concat([leading_historical_inputs, df_y_input_tas], axis=1)
Xy_train_pr_ = pd.concat([leading_historical_inputs, df_y_input_pr], axis=1)
Xy_train_pr90_ = pd.concat([leading_historical_inputs, df_y_input_pr90], axis=1)
Xy_train_dtr_ = pd.concat([leading_historical_inputs, df_y_input_dtr], axis=1)

In [ ]:
Xy_train_tas = Xy_train_tas_.to_numpy()
Xy_train_pr = Xy_train_pr_.to_numpy()
Xy_train_pr90 = Xy_train_pr90_.to_numpy()
Xy_train_dtr = Xy_train_dtr_.to_numpy()

In [ ]:
n_inp=leading_historical_inputs.shape[1]
n_iout=Xy_train_tas_.shape[1]

X_train_tas=Xy_train_tas[:,0:n_inp]
y_train_tas=Xy_train_tas[:,n_inp:n_iout]

X_train_pr=Xy_train_pr[:,0:n_inp]
y_train_pr=Xy_train_pr[:,n_inp:n_iout]

X_train_pr90=Xy_train_pr90[:,0:n_inp]
y_train_pr90=Xy_train_pr90[:,n_inp:n_iout]

X_train_dtr=Xy_train_dtr[:,0:n_inp]
y_train_dtr=Xy_train_dtr[:,n_inp:n_iout]


In [ ]:
reg0 = RandomForestRegressor(random_state=0)
reg1 = RandomForestRegressor(random_state=0)
reg2 = RandomForestRegressor(random_state=0)
reg3 = RandomForestRegressor(random_state=0)

if(RSCV==False):
    rf_tas = reg0.fit(X_train_tas,y_train_tas)
    rf_pr = reg1.fit(X_train_pr,y_train_pr)
    rf_pr90 = reg2.fit(X_train_pr90,y_train_pr90)
    rf_dtr = reg3.fit(X_train_dtr,y_train_dtr)
else:
    rf_random0 = RandomizedSearchCV(estimator = reg0, param_distributions = random_grid, n_iter = 29, cv = 3, verbose=2, n_jobs = -1)
    rf_random1 = RandomizedSearchCV(estimator = reg1, param_distributions = random_grid, n_iter = 29, cv = 3, verbose=2, n_jobs = -1)
    rf_random2 = RandomizedSearchCV(estimator = reg2, param_distributions = random_grid, n_iter = 29, cv = 3, verbose=2, n_jobs = -1)
    rf_random3 = RandomizedSearchCV(estimator = reg3, param_distributions = random_grid, n_iter = 29, cv = 3, verbose=2, n_jobs = -1)

    #n_iter = 29
    
    rf_tas = rf_random0.fit(X_train_tas,y_train_tas)
    rf_pr = rf_random1.fit(X_train_pr,y_train_pr)
    rf_pr90 = rf_random2.fit(X_train_pr90,y_train_pr90)
    rf_dtr = rf_random3.fit(X_train_dtr,y_train_dtr)

In [ ]:
if(RSCV==True):
    print(rf_tas.best_params_)
    print(rf_pr.best_params_)
    print(rf_pr90.best_params_)
    print(rf_dtr.best_params_)

In [ ]:
## Test on SSP245

test_Y = xr.open_dataset(data_path + 'test-data/outputs_ssp245_solution.nc').compute()
test_X = xr.open_dataset(data_path + 'test-data/inputs_ssp245-2.nc').compute()

tas_truth = test_Y["tas"].mean('member')
pr_truth = test_Y["pr"].mean('member') * 86400
pr90_truth = test_Y["pr90"].mean('member') * 86400
dtr_truth = test_Y["diurnal_temperature_range"].mean('member')

test_inputs = pd.DataFrame({
    "CO2": normalize_co2(test_X["CO2"].data),
    "CH4": normalize_ch4(test_X["CH4"].data)
}, index=test_X["CO2"].coords['time'].data)

### Combine with aerosol EOFs
test_inputs=pd.concat([test_inputs, 
                       bc_solver.projectField(test_X["BC"], neofs=5, eofscaling=1).to_dataframe().unstack('mode').rename(columns={i:f"BC_{i}" for i in range(5)}),
                       so2_solver.projectField(test_X["SO2"], neofs=5, eofscaling=1).to_dataframe().unstack('mode').rename(columns={i:f"_{i}" for i in range(5)}),
                       ], axis=1)


In [ ]:
m_out_t = rf_tas.predict(test_inputs)
m_out_p = rf_pr.predict(test_inputs)
m_out_p90 = rf_pr90.predict(test_inputs)
m_out_d = rf_dtr.predict(test_inputs)

m_out_tas = m_out_t.reshape(86, 96, 144)
m_out_pr = m_out_p.reshape(86, 96, 144)
m_out_pr90 = m_out_p90.reshape(86, 96, 144)
m_out_dtr = m_out_d.reshape(86, 96, 144)

In [ ]:
#xr_output['pr'].sel(time=2015).plot()

In [ ]:
#m_tas.sel(sample=35).plot(cmap="coolwarm", norm=divnorm,
#                          cbar_kwargs={"label":"Temperature change / K"})
#plt.gca().set_title("Emulated 2050")

In [ ]:
xr_output=xr.Dataset(coords={'time': test_X.time.values, 'lat': test_X.latitude.values, 'lon': test_X.longitude.values})
xr_output["tas"]=(['time', 'lat', 'lon'],  m_out_tas)
xr_output["diurnal_temperature_range"]=(['time', 'lat', 'lon'],  m_out_dtr)
xr_output["pr"]=(['time', 'lat', 'lon'],  m_out_pr)
xr_output["pr90"]=(['time', 'lat', 'lon'],  m_out_pr90)
    
#save output to netcdf 
xr_output.to_netcdf(path_output,'w')

In [ ]:
#coords={'lon': (['x', 'y'], lon),
#                    'lat': (['x', 'y'], lat),
#                    'time': pd.date_range('2014-09-06', periods=3)})
#temp=np.array([[25, 24, 20, -12],[23, 21, 22, -11]])
#xr_output["Temperature"]=(['x', 'y', 'time'],  temp)

In [ ]:
print(f"RMSE: {get_rmse(tas_truth[35], m_out_tas[35])}")
print(f"RMSE: {get_rmse(tas_truth[85], m_out_tas[85])}")
print(f"RMSE: {get_rmse(tas_truth[30:40], m_out_tas[30:40])}")
print(f"RMSE: {get_rmse(tas_truth[75:], m_out_tas[75:])}")
print(f"RMSE: {get_rmse(tas_truth[35:], m_out_tas[35:])}")
print("\n")

print(f"RMSE: {get_rmse(dtr_truth[35], m_out_dtr[35])}")
print(f"RMSE: {get_rmse(dtr_truth[85], m_out_dtr[85])}")
print(f"RMSE: {get_rmse(dtr_truth[30:40], m_out_dtr[30:40])}")
print(f"RMSE: {get_rmse(dtr_truth[75:], m_out_dtr[75:])}")
print(f"RMSE: {get_rmse(dtr_truth[35:], m_out_dtr[35:])}")
print("\n")

print(f"RMSE: {get_rmse(pr_truth[35], m_out_pr[35])}")
print(f"RMSE: {get_rmse(pr_truth[85], m_out_pr[85])}")
print(f"RMSE: {get_rmse(pr_truth[30:40], m_out_pr[30:40])}")
print(f"RMSE: {get_rmse(pr_truth[75:], m_out_pr[75:])}")
print(f"RMSE: {get_rmse(pr_truth[35:], m_out_pr[35:])}")
print("\n")

print(f"RMSE: {get_rmse(pr90_truth[35], m_out_pr90[35])}")
print(f"RMSE: {get_rmse(pr90_truth[85], m_out_pr90[85])}")
print(f"RMSE: {get_rmse(pr90_truth[30:40], m_out_pr90[30:40])}")
print(f"RMSE: {get_rmse(pr90_truth[75:], m_out_pr90[75:])}")
print(f"RMSE: {get_rmse(pr90_truth[35:], m_out_pr90[35:])}")

In [ ]:
#predict avg last 20 years

m_out_t = rf_tas.predict(test_inputs[-20:].mean().values.reshape(1,12))
m_out_p = rf_pr.predict(test_inputs[-20:].mean().values.reshape(1,12))
m_out_p90 = rf_pr90.predict(test_inputs[-20:].mean().values.reshape(1,12))
m_out_d = rf_dtr.predict(test_inputs[-20:].mean().values.reshape(1,12))

m_out_tas = m_out_t.reshape(96, 144)
m_out_pr = m_out_p.reshape(96, 144)
m_out_pr90 = m_out_p90.reshape(96, 144)
m_out_dtr = m_out_d.reshape(96, 144)

In [ ]:
tas_truth2=tas_truth[-20:].mean(dim="time")
pr_truth2=pr_truth[-20:].mean(dim="time")
pr90_truth2=pr90_truth[-20:].mean(dim="time")
dtr_truth2=dtr_truth[-20:].mean(dim="time")

print(f"RMSE: {get_rmse(tas_truth2, m_out_tas)}")
print(f"RMSE: {get_rmse(dtr_truth2, m_out_dtr)}")
print(f"RMSE: {get_rmse(pr_truth2, m_out_pr)}")
print(f"RMSE: {get_rmse(pr90_truth2, m_out_pr90)}")
